# Программирование для всех<br>(основы работы с Python)

*Алла Тамбовцева*

## Практикум 10.1. Датафреймы pandas – часть 3

* Выбор строк и столбцов с помощью `.loc` и `.iloc`
* Объединение датафреймов
* Группировка и агрегирование

In [1]:
import pandas as pd

### Загрузка и описание данных

В файле `flats-final.xlsx` два листа. На первом листе `flats` хранится информация по квартирам в Москве (выгрузка с ЦИАН по состоянию на май 2025):

* `price`: цена в рублях;
* `lprice`: логарифм цены;
* `square`: площадь квартиры, в кв. метрах;
* `rooms`: число комнат;
* `floor`: этаж;
* `mfloor`: число этажей в доме;
* `station`: станция метро;
* `metro`: доступность и расстояние до метро;
* `ametro`: шаговая доступность метро (1 – да, 0 – нет);
* `dmetro`: расстояние до метро (на транспорте или пешком), в минутах;
* `link`: ссылка на объявление;
* `add`: адрес;
* `lat`: широта;
* `lon`: долгота.

На втором листе `districts` хранится информация о районах и округах:

* `station`: станция метро;
* `district`: название района;
* `okrug`: название административного округа.

Загрузим данные из файла `flats-final.xlsx`. По умолчанию и так загружаются данные с первого листа, но для универсальности укажем индекс листа в `sheet_name`):

In [2]:
# в sheet_name может быть в название листа в кавычках

flats = pd.read_excel("flats-final.xlsx", sheet_name = 0)
flats.head()

,price,lprice,square,rooms,floor,mfloor,station,metro,ametro,dmetro,link,add,lat,lon
0,12000000,16.300417,60.0,3,1,9,метро Аннино,6 минут на транспорте,0,6,https://www.cian.ru/sale/flat/314948168/,"Чертановская ул., 66К2",55.596930,37.585664
1,12400000,16.333207,60.1,3,1,9,метро Аннино,6 минут на транспорте,0,6,https://www.cian.ru/sale/flat/313019655/,"Чертановская ул., 66К2",55.596930,37.585664
2,11499999,16.257858,37.9,1,16,20,метро Аннино,19 минут пешком,1,19,https://www.cian.ru/sale/flat/315677385/,"ул. Газопровод, 15",55.589274,37.612650
3,11950000,16.296242,40.0,1,2,17,метро Аннино,19 минут пешком,1,19,https://www.cian.ru/sale/flat/316242727/,"ул. Газопровод, 15",55.589274,37.612650
4,36000000,17.399029,120.0,4,1,9,метро Аннино,7 минут на транспорте,0,7,https://www.cian.ru/sale/flat/309425036/,"Старокачаловская ул., 14",55.569708,37.587596


In [3]:
# 10054 строки и 14 столбцов

flats.shape

(10054, 14)

Выберем сразу несколько столбцов списком и выведем по ним описательные статистики с помощью метода `.describe()`:

In [4]:
flats[["price", "square", "rooms"]].describe()

,price,square,rooms
count,1.005400e+04,10054.000000,10054.000000
mean,4.781103e+07,74.339765,2.362443
std,8.397270e+07,48.774702,1.045435
min,3.200000e+06,10.700000,1.000000
25%,1.450000e+07,44.000000,2.000000
50%,2.192500e+07,60.000000,2.000000
75%,4.300000e+07,84.000000,3.000000
max,1.732171e+09,779.400000,5.000000


> Выберите строки, которые соответствуют квартирам с площадью более 300 квадратных метров.

In [5]:
f01 = flats[flats["square"] > 300]
print(f01.shape) # таких 46

(46, 14)


> Выберите строки, которые соответствуют квартирам с максимальной ценой.

In [6]:
flats[flats["price"] == flats["price"].max()]

,price,lprice,square,rooms,floor,mfloor,station,metro,ametro,dmetro,link,add,lat,lon
4487,1732170835,21.272641,392.1,5,5,5,метро Площадь Революции,16 минут пешком,1,16,https://www.cian.ru/sale/flat/300661842/,"Газетный пер., 13С1",55.758894,37.609398


Также загрузим данные со второго листа:

In [7]:
okrs = pd.read_excel("flats-final.xlsx", sheet_name = 1)
okrs.tail()

,district,station,okrug
141,район Тропарево-Никулино,метро Юго-Западная,Западный АО
142,район Хамовники,метро Фрунзенская,Центральный АО
143,район Хорошево-Мневники,метро Хорошёвская,Северно-Западный АО
144,район Хорошево-Мневники,метро Полежаевская,Северно-Западный АО
145,район Чертаново Южное,метро Улица Академика Янгеля,Южный АО


Опишем и их тоже:

In [8]:
okrs.describe()

,district,station,okrug
count,146,146,146
unique,55,146,10
top,Новомосковский административный округ,метро Саларьево,Центральный АО
freq,11,1,29


In [9]:
okrs["okrug"].value_counts()

Центральный АО         29
Юго-Западный АО        21
Южный АО               20
Западный АО            15
Восточный АО           12
Новомосковский АО      11
Юго-Восточный АО       11
Северно-Западный АО    10
Северо-Восточный АО     9
Северный АО             8
Name: okrug, dtype: int64

### Выбор строк и столбцов с помощью `.loc` и `.iloc`

Для более универсального выбора строк и столбцов на датафреймах `pandas` определены методы `.loc` и `.iloc`. Название метода `.loc` происходит от *location*, а метода `.iloc` – от *index location*, поэтому:

* `.loc` выбирает элементы по индексам строк (целые числа или текст) и названиям столбцов (текст);
* `.iloc` выбирает элементы по индексам строк (целые числа) и индексам столбцов (целые числа).

Так как эти методы являются дополнением к операторам `[]` для фильтрации, при их вызове тоже используются квадратные скобки, а не круглые. На первом месте в квадратных скобках указываются идентификаторы строк, на втором – идентификаторы столбцов, например:

    flats.loc[0, "price"]
    flats.iloc[0, 0]

>Используя сначала метод `.loc`, а затем `.iloc`, выведите на экран: площадь квартиры с индексом 50. 

In [10]:
# перечень названий столбцов для наглядности
print(flats.columns)

Index(['price', 'lprice', 'square', 'rooms', 'floor', 'mfloor', 'station',
       'metro', 'ametro', 'dmetro', 'link', 'add', 'lat', 'lon'],
      dtype='object')


In [11]:
print(flats.loc[50, "square"])
print(flats.iloc[50, 2])

67.2
67.2


> Подумайте, как запросить элементы сразу по нескольким индексам/названиям. Используя `.loc`, выведите значение цены и площади для квартиры с индексом 20. Используя `.iloc`, выведите цену и площадь квартир с индексами 100, 200, 300.

In [12]:
flats.loc[20, ["price", "square", "add"]]

price               18900000
square                  45.7
add       Варшавское ш., 168
Name: 20, dtype: object

**Комментарий.** Pandas выбрал строку с номером 20, из нее – нужный фрагмент с ценой, площадью и адресом:

|    |   price | square       | add |
|---:|--------:|:-------------|:-------|
| 20 |  18900000 | 45.7 | Варшавское ш., 168 |

А затем транспонировал результат – записал строку в виде столбца. Выше в выдаче мы видим последовательность `pandas Series`, столбец с тремя записями: индекс `price` и значение 18900000, индекс `square` и значение 45.7, индекс `add` и значение `Варшавское ш., 168`. В названии `Name` сохранен номер строки, которую мы выбрали изначально. А вот с типом столбца все любопытно. В `dtype` написано `object`, Python намекает, что в столбце сохранены текстовые значения. В теории так и должно быть – если в столбце есть и текст, и числа, текстовый тип `object`, как более сильный, должен вытеснить все остальные. Однако в действительности этого не происходит. Хотя Python и написал `object`, значения в последовательности сохранили свои типы:

In [13]:
res = flats.loc[20, ["price", "square", "add"]]

# цена осталась числом типа int
# площадь осталась числом типа float

print(type(res["price"]))
print(type(res["square"]))
print(type(res["add"]))

<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>


Это сделано для удобства – ценность `.loc` и `.iloc` была бы сомнительной, если бы после фильтрации с их помощью пришлось заново «восстанавливать» все типы, чтобы полноценно работать с выбранными фрагментами таблицы.

In [14]:
# строки с номерами 100, 200, 300
# столбцы с номерами 0 и 2

flats.iloc[[100, 200, 300], [0, 2]]

,price,square
100,25990000,72.7
200,33000000,82.0
300,56000000,138.0


Как быть, если нужные нам индексы или названия следуют друг за другом? Воспользоваться срезами (*slices*), как на списках или массивах!

> Выведите на экран площадь, число комнат и число этажей в доме для квартир с индексами от 600 до 614 включительно. Используйте сначала `.loc`, а затем – `.iloc`.

In [15]:
# loc - правая граница среза включается
# строки с номерами от 600 до 614 включительно,
# столбцы от square до mfloor включительно

flats.loc[600:614, "square":"mfloor"]

,square,rooms,floor,mfloor
600,48.8,3,8,9
601,45.0,2,6,9
602,47.0,2,3,9
603,58.8,3,8,9
604,44.0,2,7,9
605,32.0,1,8,9
606,43.4,2,8,9
607,62.8,4,9,9
608,64.8,3,9,9
609,32.0,1,9,9


In [16]:
# iloc - правая граница среза НЕ включается
# строки с номерами от 600 до 615, исключая 615
# столбцы с номерами от 2 до 6, исключая 6

flats.iloc[600:615, 2:6]

,square,rooms,floor,mfloor
600,48.8,3,8,9
601,45.0,2,6,9
602,47.0,2,3,9
603,58.8,3,8,9
604,44.0,2,7,9
605,32.0,1,8,9
606,43.4,2,8,9
607,62.8,4,9,9
608,64.8,3,9,9
609,32.0,1,9,9


**Комментарий.** Так как метод `.iloc` работает исключительно с целочисленными индексами, он «наследует» поведение числовых срезов и функции `range()` в базовом Python – исключает правую границу диапазона, который мы выбираем. Метод `.loc` работает с текстовыми значениями в том числе, поэтому в нем срезы работают несколько иначе.

### Объединение датафреймов

Для дальнейшей работы логичным будет объединить данные, взятые с двух листов файла Excel, в одну таблицу. Основание для объединения у нас есть – и в первой таблице, и во второй названия станций метро сохранены в столбце `station`.

In [17]:
print(flats.columns)
print(okrs.columns)

Index(['price', 'lprice', 'square', 'rooms', 'floor', 'mfloor', 'station',
       'metro', 'ametro', 'dmetro', 'link', 'add', 'lat', 'lon'],
      dtype='object')
Index(['district', 'station', 'okrug'], dtype='object')


«Доклеим» к датафрейму `okrs` датафрейм `okrs`, объединив их по общему столбцу с названием `station`:

In [18]:
df = flats.merge(okrs, on = "station")
df.head()

,price,lprice,square,rooms,floor,mfloor,station,metro,ametro,dmetro,link,add,lat,lon,district,okrug
0,12000000,16.300417,60.0,3,1,9,метро Аннино,6 минут на транспорте,0,6,https://www.cian.ru/sale/flat/314948168/,"Чертановская ул., 66К2",55.596930,37.585664,район Северное Бутово,Юго-Западный АО
1,12400000,16.333207,60.1,3,1,9,метро Аннино,6 минут на транспорте,0,6,https://www.cian.ru/sale/flat/313019655/,"Чертановская ул., 66К2",55.596930,37.585664,район Северное Бутово,Юго-Западный АО
2,11499999,16.257858,37.9,1,16,20,метро Аннино,19 минут пешком,1,19,https://www.cian.ru/sale/flat/315677385/,"ул. Газопровод, 15",55.589274,37.612650,район Северное Бутово,Юго-Западный АО
3,11950000,16.296242,40.0,1,2,17,метро Аннино,19 минут пешком,1,19,https://www.cian.ru/sale/flat/316242727/,"ул. Газопровод, 15",55.589274,37.612650,район Северное Бутово,Юго-Западный АО
4,36000000,17.399029,120.0,4,1,9,метро Аннино,7 минут на транспорте,0,7,https://www.cian.ru/sale/flat/309425036/,"Старокачаловская ул., 14",55.569708,37.587596,район Северное Бутово,Юго-Западный АО


Теперь можем работать с датафреймом `df`, в котором хранится полная информация о квартирах.

### Группировка и агрегирование

Сгруппируем строки по административным округам – укажем основание группировки в качестве аргумента в специальном методе `.groupby()`:

In [19]:
df.groupby("okrug")

Объект специального типа `DataFrameGroupBy` от нас скрыт. Преобразуем его в более понятный список и посмотрим на первые два элемента:

In [20]:
# print(list(df.groupby("okrug")))

**Комментарий.** Результат `.groupby()` – объект специального типа `DataFrameGroupBy`, содержимое которого от нас скрыто и временно хранится в какой-то ячейке памяти. Если преобразовать этот объект в более понятный список через функцию `list()`, получим список из 10 пар, на первом месте в каждой паре – название округа (группировка по `okrug`), на втором – датафрейм со строками, соответствующими этому округу.

> Пользуясь тем, что результат группировки представляет собой объект, похожий по структуре на список пар, сохраните в отдельные CSV-файлы строки, соответствующие каждому округу. Воспользуйтесь методом `.to_csv()` и назовите полученные файлы по названию административного округа, например, `Восточный АО.csv`.

In [21]:
# проходим в цикле по набору пар 
# (название группы name, датафрейм для группы tab)
# выгружаем tab с названием name + расширение .csv

for name, tab in df.groupby("okrug"):
    tab.to_csv(name + ".csv")

Теперь в рабочей папке, которая отображается в *Home* в Jupyter, должны добавиться десять CSV-файлов (`Восточный.csv`, `Западный.csv` и так далее до `Южный.csv`), в каждом – строки из `df`, соответствующие каждому округу.

Если нужно, можем из результата группировки забрать датафрейм для конкретной группы – вызвать метод `.get_group()` и указать название группы:

In [22]:
# строки df для Южного округа, okrug = Южный

df.groupby("okrug").get_group("Южный")

KeyError: 'Южный'

**Дополнительно.** Как получить перечень файлов в рабочей папке? Импортировать модуль `os` для работы с операционной системой и через функцию `listdir()` запросить список названий файлов рабочей папки:

In [23]:
import os
sorted(os.listdir())

['.DS_Store',
 '.ipynb_checkpoints',
 'flats-final.xlsx',
 'flats_agg.csv',
 'flats_with_district.csv',
 'maps.ipynb',
 'practice-10-01-solved.ipynb',
 'prepare.ipynb',
 'Восточный АО.csv',
 'Западный АО.csv',
 'Москва_Moscow.geojson',
 'Новомосковский АО.csv',
 'Северно-Западный АО.csv',
 'Северный АО.csv',
 'Северо-Восточный АО.csv',
 'Центральный АО.csv',
 'Юго-Восточный АО.csv',
 'Юго-Западный АО.csv',
 'Южный АО.csv']

Посмотрим на примеры агрегирования – вычисление разных числовых характеристик по группам:

In [24]:
# группируем по округу, выбираем в каждой группе столбец 
# price, выводим статистики по ценам

df.groupby("okrug")["price"].describe()

,count,mean,std,min,25%,50%,75%,max
okrug,,,,,,,,
Восточный АО,648.0,1.742677e+07,1.017463e+07,6885000.0,11500000.0,14600000.0,1.950000e+07,1.000000e+08
Западный АО,1203.0,5.215860e+07,6.080670e+07,4700000.0,22750000.0,36599000.0,5.900000e+07,9.581795e+08
Новомосковский АО,938.0,1.435078e+07,5.348070e+06,3900000.0,10900000.0,13394950.0,1.650000e+07,5.500000e+07
Северно-Западный АО,631.0,3.027138e+07,3.677962e+07,5950000.0,14500000.0,20500000.0,3.290000e+07,6.612392e+08
Северный АО,485.0,2.934186e+07,2.560968e+07,7900000.0,15500000.0,20700000.0,3.150000e+07,1.850000e+08
Северо-Восточный АО,884.0,2.600600e+07,2.656395e+07,5500000.0,14700750.0,19799500.0,2.808068e+07,5.500000e+08
Центральный АО,1900.0,1.165377e+08,1.418806e+08,4500000.0,31000000.0,67000000.0,1.438312e+08,1.732171e+09
Юго-Восточный АО,845.0,2.042814e+07,1.734575e+07,4850000.0,12100000.0,15345000.0,2.270000e+07,2.750000e+08
Юго-Западный АО,1109.0,3.889016e+07,9.098858e+07,3200000.0,13999000.0,18900000.0,3.054000e+07,1.147000e+09


In [25]:
# группируем по округу, выбираем в каждой группе столбцы 
# price и square, выводим статистики для них

df.groupby("okrug")[["price", "square"]].describe()

price                                         \
                      count          mean           std        min   
okrug                                                                
Восточный АО          648.0  1.742677e+07  1.017463e+07  6885000.0   
Западный АО          1203.0  5.215860e+07  6.080670e+07  4700000.0   
Новомосковский АО     938.0  1.435078e+07  5.348070e+06  3900000.0   
Северно-Западный АО   631.0  3.027138e+07  3.677962e+07  5950000.0   
Северный АО           485.0  2.934186e+07  2.560968e+07  7900000.0   
Северо-Восточный АО   884.0  2.600600e+07  2.656395e+07  5500000.0   
Центральный АО       1900.0  1.165377e+08  1.418806e+08  4500000.0   
Юго-Восточный АО      845.0  2.042814e+07  1.734575e+07  4850000.0   
Юго-Западный АО      1109.0  3.889016e+07  9.098858e+07  3200000.0   
Южный АО             1411.0  3.902042e+07  5.740636e+07  6200000.0   

                                                                         \
                            25%         50%           75%           max   
okrug                                                                     
Восточный АО         11500000.0  14600000.0  1.950000e+07  1.000000e+08   
Западный АО          22750000.0  36599000.0  5.900000e+07  9.581795e+08   
Новомосковский АО    10900000.0  13394950.0  1.650000e+07  5.500000e+07   
Северно-Западный АО  14500000.0  20500000.0  3.290000e+07  6.612392e+08   
Северный АО          15500000.0  20700000.0  3.150000e+07  1.850000e+08   
Северо-Восточный АО  14700750.0  19799500.0  2.808068e+07  5.500000e+08   
Центральный АО       31000000.0  67000000.0  1.438312e+08  1.732171e+09   
Юго-Восточный АО     12100000.0  15345000.0  2.270000e+07  2.750000e+08   
Юго-Западный АО      13999000.0  18900000.0  3.054000e+07  1.147000e+09   
Южный АО             15000000.0  21700000.0  4.000000e+07  8.000000e+08   

                     square                                              \
                      count        mean        std   min     25%    50%   
okrug                                                                     
Восточный АО          648.0   57.270216  25.331102  18.0  39.375  51.90   
Западный АО          1203.0   82.777307  51.345150  15.7  50.850  68.80   
Новомосковский АО     938.0   54.893390  18.715873  19.6  39.500  52.25   
Северно-Западный АО   631.0   68.519968  49.607837  16.3  42.100  56.30   
Северный АО           485.0   64.403505  31.410761  25.0  41.600  56.00   
Северо-Восточный АО   884.0   63.556674  33.897311  15.2  42.500  57.40   
Центральный АО       1900.0  106.287947  66.772123  16.9  59.000  87.30   
Юго-Восточный АО      845.0   58.260710  25.563951  14.0  40.200  52.80   
Юго-Западный АО      1109.0   70.325158  47.448457  10.7  43.200  58.60   
Южный АО             1411.0   70.450673  41.684969  15.2  43.600  58.10   

                                     
                         75%    max  
okrug                                
Восточный АО          65.000  172.1  
Западный АО           96.350  593.3  
Новомосковский АО     64.725  135.0  
Северно-Западный АО   75.700  779.4  
Северный АО           75.100  185.0  
Северо-Восточный АО   74.000  482.5  
Центральный АО       135.000  493.8  
Юго-Восточный АО      70.700  322.0  
Юго-Западный АО       76.800  500.0  
Южный АО              82.000  430.0

In [26]:
# группируем по округу, выбираем в каждой группе столбцы 
# price и square, выводим для них только среднее mean()

df.groupby("okrug")[["price", "square"]].mean()

,price,square
okrug,,
Восточный АО,1.742677e+07,57.270216
Западный АО,5.215860e+07,82.777307
Новомосковский АО,1.435078e+07,54.893390
Северно-Западный АО,3.027138e+07,68.519968
Северный АО,2.934186e+07,64.403505
Северо-Восточный АО,2.600600e+07,63.556674
Центральный АО,1.165377e+08,106.287947
Юго-Восточный АО,2.042814e+07,58.260710
Юго-Западный АО,3.889016e+07,70.325158


In [27]:
# группируем по округу, выбираем в каждой группе столбцы 
# price и square, выводим для них только медиану median()

df.groupby("okrug")[["price", "square"]].median()

,price,square
okrug,,
Восточный АО,14600000.0,51.90
Западный АО,36599000.0,68.80
Новомосковский АО,13394950.0,52.25
Северно-Западный АО,20500000.0,56.30
Северный АО,20700000.0,56.00
Северо-Восточный АО,19799500.0,57.40
Центральный АО,67000000.0,87.30
Юго-Восточный АО,15345000.0,52.80
Юго-Западный АО,18900000.0,58.60


In [28]:
# группируем по округу, выбираем в каждой группе столбцы 
# price и square, выводим для них только минимум

df.groupby("okrug")[["price", "square"]].min()

,price,square
okrug,,
Восточный АО,6885000,18.0
Западный АО,4700000,15.7
Новомосковский АО,3900000,19.6
Северно-Западный АО,5950000,16.3
Северный АО,7900000,25.0
Северо-Восточный АО,5500000,15.2
Центральный АО,4500000,16.9
Юго-Восточный АО,4850000,14.0
Юго-Западный АО,3200000,10.7


In [29]:
# группируем по округу, выбираем в каждой группе столбцы 
# price и square, выводим для них только максимум

df.groupby("okrug")[["price", "square"]].max()

,price,square
okrug,,
Восточный АО,100000000,172.1
Западный АО,958179500,593.3
Новомосковский АО,55000000,135.0
Северно-Западный АО,661239200,779.4
Северный АО,185000000,185.0
Северо-Восточный АО,550000000,482.5
Центральный АО,1732170835,493.8
Юго-Восточный АО,275000000,322.0
Юго-Западный АО,1147000000,500.0


In [30]:
# если хотим сразу несколько статистик, например, 
# среднее mean и стандартное отклонение std,
# названия соответствующих методов можно перечислить
# списком внутри .agg (от aggregate)

df.groupby("okrug")[["lprice", "square"]].agg(["mean", "std"])

lprice                square           
                          mean       std        mean        std
okrug                                                          
Восточный АО         16.568977  0.419270   57.270216  25.331102
Западный АО          17.462826  0.719168   82.777307  51.345150
Новомосковский АО    16.421976  0.331353   54.893390  18.715873
Северно-Западный АО  16.960900  0.636014   68.519968  49.607837
Северный АО          16.969165  0.610826   64.403505  31.410761
Северо-Восточный АО  16.887000  0.542208   63.556674  33.897311
Центральный АО       18.067444  0.989783  106.287947  66.772123
Юго-Восточный АО     16.669760  0.505498   58.260710  25.563951
Юго-Западный АО      16.950231  0.763341   70.325158  47.448457
Южный АО             17.088271  0.764677   70.450673  41.684969

> Подумайте, как сгруппировать строки сразу по нескольким основаниям (вспомните сортировку по нескольким основания). Сгруппируйте квартиры по округу и шаговой доступности станции метро, выведите среднюю цену квартиры для каждой группы.

In [31]:
# перечень оснований группировки – в виде списка

df.groupby(["okrug", "district", "ametro"])["price"].mean()

okrug         district                      ametro
Восточный АО  район Богородское             0         1.600764e+07
                                            1         2.166453e+07
              район Вешняки                 0         1.727973e+07
                                            1         1.605107e+07
              район Восточное Измайлово     0         2.448109e+07
                                                          ...     
Южный АО      район Зябликово               0         1.353500e+07
                                            1         1.509092e+07
              район Орехово-Борисово Южное  0         1.099900e+07
              район Чертаново Южное         0         8.700000e+06
                                            1         1.498027e+07
Name: price, Length: 104, dtype: float64

> Сгруппируйте квартиры по району, вычислите медианную цену и площадь квартиры для каждого района. Сохраните результат агрегирования и экспортируйте полученный датафрейм в файл `flats_agg.csv`. Он понадобится нам для практикума 10.2.

In [32]:
data = df.groupby("district")[["price", "square"]].median()
data.to_csv("flats_agg.csv")

P.S. Не понадобится – заменила файл на `flats_with_district.csv`, чтобы на картах не было «белых пятен». Чтобы не грузить очень большой файл, в `flats-final.xlsx` для текущего практикума выгрузила относительно небольшую случайную выборку квартир, поэтому не все районы представлены в данных (в начале этого практикума видно, что уникальных значений в `district` 55, хотя районов в Москве сильно больше).